In [1]:
%reset -f

In [2]:
import ee
ee.Authenticate()


In [3]:
'https://gee-community-catalog.org/projects/polaris/'

import ee

ee.Initialize(project='ee-andrewfullhart')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')
first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

polar_ic = ee.ImageCollection('projects/sat-io/open-datasets/polaris/bd_mean')

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

label_order = ee.List(['bd', 'clay', 'ksat', 'om', 'ph', 'sand', 'silt', 'theta_r', 'theta_s'])

im = polar_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
];

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
];

proj = im.projection();


bd = ee.ImageCollection('projects/sat-io/open-datasets/polaris/bd_mean')
clay = ee.ImageCollection('projects/sat-io/open-datasets/polaris/clay_mean')
ksat = ee.ImageCollection('projects/sat-io/open-datasets/polaris/ksat_mean')
om = ee.ImageCollection('projects/sat-io/open-datasets/polaris/om_mean')
ph = ee.ImageCollection('projects/sat-io/open-datasets/polaris/ph_mean')
sand = ee.ImageCollection('projects/sat-io/open-datasets/polaris/sand_mean')
silt = ee.ImageCollection('projects/sat-io/open-datasets/polaris/silt_mean')
theta_r = ee.ImageCollection('projects/sat-io/open-datasets/polaris/theta_r_mean')
theta_s = ee.ImageCollection('projects/sat-io/open-datasets/polaris/theta_s_mean')

bd = bd.reduce(ee.Reducer.mean())
clay = clay.reduce(ee.Reducer.mean())
ksat = ksat.reduce(ee.Reducer.mean())
om = om.reduce(ee.Reducer.mean())
ph = ph.reduce(ee.Reducer.mean())
sand = sand.reduce(ee.Reducer.mean())
silt = silt.reduce(ee.Reducer.mean())
theta_r = theta_r.reduce(ee.Reducer.mean())
theta_s = theta_s.reduce(ee.Reducer.mean())

map_ic = ee.ImageCollection([bd, clay, ksat, om, ph, sand, silt, theta_r, theta_s])

def clip_fn(im):
  im = im.setDefaultProjection('EPSG:4326', transform)
  im = im.reproject(crs=proj.crs(), crsTransform=transform_new)
  clip_im = ee.Image(im).clip(area_shp)
  return clip_im

map_ic = ee.ImageCollection(map_ic.map(clip_fn))
map_ic_list = map_ic.toList(map_ic.size())

for i in range(map_ic.size().getInfo()):

  eimage = ee.Image(map_ic_list.get(i))
  label = ee.String(label_order.get(i)).getInfo()

  task = ee.batch.Export.image.toDrive(
      image=eimage,
      description='Polaris_' + label,
      folder='GEE_Downloads',
      region=area_shp,
      scale=scale,
      crs='EPSG:4326',
      maxPixels=1e13)

  task.start()


